In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
TRAIN_DIR = '../dataset/train'
TEST_DIR = '../dataset/test'

In [3]:
import os

In [4]:
MAP_CHARACTERS = {}

for idx, training_class in enumerate(os.listdir(TRAIN_DIR)):
    MAP_CHARACTERS[idx] = training_class

In [5]:
BATCH_SIZE = 32

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255., validation_split=.2)

train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='training'
)

val_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='validation'
)

Found 13210 images belonging to 32 classes.
Found 3286 images belonging to 32 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_directory(
    directory=TEST_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True
)

Found 4142 images belonging to 32 classes.


In [8]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [9]:
tf.keras.backend.clear_session()

In [10]:
mbnet = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet')
mbnet.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [11]:
inputs = tf.keras.layers.Input((224, 224, 3))

In [12]:
x = mbnet(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

In [13]:
outputs = tf.keras.layers.Dense(32, activation='softmax')(x)

In [14]:
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, None, None, 1280)  2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                81984     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
Total params: 2,342,048
Trainable params: 84,064
Non-trainable params: 2,257,984
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
          validation_data=val_generator,
          epochs=10)

Epoch 1/10
413/413 [==============================] - 138s 331ms/step - loss: 2.3927 - accuracy: 0.3524 - val_loss: 1.6896 - val_accuracy: 0.5399
Epoch 2/10
272/413 [==================>...........] - ETA: 38s - loss: 1.2347 - accuracy: 0.6696